Neural Collaborative Filtering

In [1]:
# imports

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler

from torch.utils.data import TensorDataset, DataLoader

Load Data

In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-100k.zip


In [ ]:
!unzip ml-100k.zip

In [4]:
!rm ml-100k.zip
%cd ml-100k

rm: ml-100k.zip: No such file or directory
/Users/darylfung/programming/pytorch-recommender-system/notebooks/ml-100k


In [5]:
# read csv file
data = pd.read_csv('u.data', '\t', names=['user', 'item', 'rating', 'timestamp'])

In [5]:
data.head

<bound method NDFrame.head of        user  item  rating  timestamp
0       196   242       3  881250949
1       186   302       3  891717742
2        22   377       1  878887116
3       244    51       2  880606923
4       166   346       1  886397596
...     ...   ...     ...        ...
99995   880   476       3  880175444
99996   716   204       5  879795543
99997   276  1090       1  874795795
99998    13   225       2  882399156
99999    12   203       3  879959583

[100000 rows x 4 columns]>

Create network

In [16]:
class Embedding(nn.Module):
    def __init__(self, num_users, num_items, embedding_dims):
        super(Embedding, self).__init__()
        self.user_matrix = nn.Embedding(num_users, embedding_dims)
        self.item_matrix = nn.Embedding(num_items, embedding_dims)

        self.user_matrix_mlp = nn.Embedding(num_users, embedding_dims)
        self.item_matrix_mlp = nn.Embedding(num_items, embedding_dims)
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dims * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 64))

        self.fc = nn.Linear(embedding_dims + 64, 1)

    def forward(self, user, item):
        if type(user) != torch.Tensor:
            user = torch.tensor(user)
        if type(item) != torch.Tensor:
            item = torch.tensor(item)

        user_matrix = self.user_matrix(user)
        item_matrix = self.item_matrix(item)
        gmf_output = user_matrix * item_matrix

        mlp_user_matrix = self.user_matrix_mlp(user)
        mlp_item_matrix = self.item_matrix_mlp(item)
        mlp_matrix = torch.cat([mlp_user_matrix, mlp_item_matrix], dim=1)
        mlp_output = self.mlp(mlp_matrix)

        total_output = torch.cat([gmf_output, mlp_output], dim=1)
        outputs = self.fc(total_output)

        return outputs

num_users = data.max()['user']+1
num_items = data.max()['item']+1

network = Embedding(int(num_users), int(num_items), 64)
predicted_rating = network(data['user'][0:3], data['item'][0:3])

Test network

In [8]:
num_users = data.max()['user']+1
num_items = data.max()['item']+1

network = Embedding(int(num_users), int(num_items), 64)
predicted_rating = network(data['user'][0:3], data['item'][0:3])

Create loss functon and optimizer

In [17]:
criterion = nn.MSELoss()
optim = torch.optim.Adam(network.parameters(), lr=0.001)

In [18]:
dataset = TensorDataset(torch.FloatTensor(data.values))
data_loader = DataLoader(dataset, 128, shuffle=False)

for i in range(5):
    for train_x in data_loader:
        optim.zero_grad()
        user = train_x[0][:, 0].long()
        item = train_x[0][:, 1].long()
        ratings = train_x[0][:, 2]
        predicted_rating = network(user, item)

        loss = criterion(predicted_rating, ratings)
        loss.backward()
        optim.step()
    print(loss)

/Users/darylfung/opt/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/Users/darylfung/opt/anaconda3/envs/py36/lib/python3.6/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


tensor(1.4350, grad_fn=<MseLossBackward>)
tensor(1.3135, grad_fn=<MseLossBackward>)
tensor(1.2392, grad_fn=<MseLossBackward>)
tensor(1.2061, grad_fn=<MseLossBackward>)
tensor(1.1863, grad_fn=<MseLossBackward>)


In [31]:
predicted_rating = network(data['user'][1:2].values, data['item'][5:6].values)